In [329]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None

In [330]:
def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0





In [331]:
def check_first_error(df:pd.DataFrame,name_file):
    """
    Функция для проверки гр. 09 и гр. 10 < гр. 08 
    """
    
    df['Сумма'] = df['09'] + df['10']
    # Проводим проверку
    df['Результат'] = df['08'] >= df['Сумма']
    # заменяем булевые значения на понятные
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно' )
     # получаем датафрейм с ошибками и извлекаем индекс
    df=df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:x +9,raw_lst_index))
    temp_error_df['Номер строки с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = 'гр. 09 и гр. 10 < гр. 08 '
    return temp_error_df

In [332]:

def check_second_error(df:pd.DataFrame,name_file):
    """    
    Функция для проверки правильности введеденных данных
    (гр. 07= гр.08 + сумма(с гр.11 по гр.32))
    :param df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    # конвертируем в инт
    all_sum_cols = list(df)
    # удаляем колонки 07, 09, 10
    all_sum_cols.remove('07')
    all_sum_cols.remove('09')
    all_sum_cols.remove('10')
    # получаем сумму колонок 08, 11:32
    df['Сумма'] = df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    df['Результат'] = df['07'] == df['Сумма']
    # заменяем булевые значения на понятные
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df=df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:x +9,raw_lst_index))
    temp_error_df['Номер строки с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = 'Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по гр.32)'
    return temp_error_df


In [333]:
def check_third_error(df:pd.DataFrame,name_file):
    """    
    Функция для проверки правильности введеденных данных
    стр. 06 = стр. 02 + стр. 04
    :param df: копия датафрейма с данными из файла поо
    :return:датафрейм с ошибками
    """
    df = pd.DataFrame(columns=['02','04','06',])

    # Добавляем данные в датафрейм
    df['02'] = temp_df.iloc[1,:]
    df['04'] = temp_df.iloc[3,:]
    df['06'] = temp_df.iloc[5,:]
    df['Сумма'] = df['02'] + df['04']
    df['Результат'] = df['06'] == df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')

    df=df[df['Результат'] == 'Неправильно'].reset_index()
    temp_error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist() # делаем список

    temp_error_df['Номер строки с ошибкой'] = raw_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = ' Не совпадают суммы стр. 06 = стр. 02 + стр. 04 '
    return temp_error_df

    

In [334]:
# создаем словарь верхнего уровня для каждого поо
high_level_dct = {}
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])

primal_df = pd.read_excel('data/poo/Брит - 2022.xlsx',skiprows=7,dtype=str)
primal_df.columns = list(map(str, primal_df.columns))
name_file = 'БРИТ'
# Заполняем пока пропуски в 15 ячейке для каждой специальности
primal_df['06'] = primal_df['06'].fillna('15 ячейка')
# Проводим проверку на корректность данных, отправляем копию датафрейма
# check_error(df.iloc[:,6:32], error_df)


In [335]:
df = primal_df.iloc[:,6:32]
df = df.applymap(check_data)

In [336]:
# получаем количество датафреймов
quantity = df.shape[0] // 15
# счетчик для обработанных датафреймов
border = 0

In [337]:
for i in range(1,quantity+1):
    temp_df = df.iloc[border:border +15,:]
    # Проводим проверку гр. 09 и гр. 10 <= гр. 08
    first_error_df = check_first_error(temp_df.copy(),name_file)
     # добавляем результат проверки в датафрейм
    error_df = pd.concat([error_df,first_error_df],axis=0,ignore_index=True)
        
    
    # Проводим проверку гр. 07= гр.08 + сумма(с гр.11 по гр.32)
    second_error_df = check_second_error(temp_df.copy(),name_file)
    # добавляем результат проверки в датафрейм
    error_df = pd.concat([error_df,second_error_df],axis=0,ignore_index=True)
    
    # Проводим проверку стр. 06 = стр. 02 + стр. 04
    third_error_df = check_third_error(temp_df.copy(),name_file)
    # добавляем результат проверки в датафрейм
    error_df = pd.concat([error_df,third_error_df],axis=0,ignore_index=True)
    
    
    
    
    
    
    
    
    # прибавляем border
    border += 15

In [338]:
error_df

,Название файла,Номер строки с ошибкой,Описание ошибки
0,БРИТ,9,гр. 09 и гр. 10 < гр. 08
1,БРИТ,10,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
2,БРИТ,11,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
3,БРИТ,12,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
4,БРИТ,14,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
5,БРИТ,08,Не совпадают суммы стр. 06 = стр. 02 + стр. 04
6,БРИТ,255,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
7,БРИТ,258,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...


In [339]:
temp_df = df.iloc[border:border +15,:]

In [340]:
temp_df.head(6)

,07,08,09,10,11,12,13,14,15,16,...,23,24,25,26,27,28,29,30,31,32


In [341]:
trash_df = pd.DataFrame(columns=['02','04','06',])

In [342]:
# Добавляем данные в датафрейм
trash_df['02'] = temp_df.iloc[1,:]
trash_df['04'] = temp_df.iloc[3,:]
trash_df['06'] = temp_df.iloc[5,:]
trash_df['Сумма'] = trash_df['02'] + trash_df['04']
trash_df['Результат'] = trash_df['06'] == trash_df['Сумма']
trash_df['Результат'] = trash_df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')


IndexError: single positional indexer is out-of-bounds

In [ ]:
df=trash_df[trash_df['Результат'] == 'Неправильно'].reset_index()
temp_error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])
# обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
raw_lst_index = df['index'].tolist() # делаем список

temp_error_df['Номер строки с ошибкой'] = raw_lst_index
temp_error_df['Название файла'] = name_file
temp_error_df['Описание ошибки'] = ' Не совпадают суммы стр. 06 = стр. 02 + стр. 04 '


In [ ]:
temp_error_df